In [2]:
%%bash
# Install required packages
pip install pandas
pip install sentence-transformers

#Had to add these to make it work locally
pip install matplotlib
pip install seaborn
pip install sentence_transformers
pip install nltk

In [3]:
import os
import json
import re
from typing import List, Tuple, Dict, Union
from pprint import pprint
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize
from sentence_transformers import SentenceTransformer
from sentence_transformers import SentenceTransformer
import torch
from torch.utils.data import TensorDataset, DataLoader
import torch.nn as nn
from torch.optim import Adam
import numpy as np
import importlib
from collections import Counter

# import QSFNN
# importlib.reload(QSFNN)

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {DEVICE} device")

Using cuda device


In [4]:
parent_dir = os.path.dirname(os.path.abspath("__file__"))
data_dir = os.path.join(parent_dir, "../../data")
print(data_dir)

/home/ameli/cse517p-project/src/data_parsing/../../data


In [5]:
raw_dataset = pd.read_csv(
    f"{data_dir}/raw_data/nasa_transcripts.csv"
)

In [6]:
raw_dataset.head()

mission      document speaker_code  \
0  apollo_17  AS17_TEC.pdf          CDR   
1  apollo_17  AS17_TEC.pdf          CDR   
2  apollo_17  AS17_TEC.pdf           CC   
3  apollo_17  AS17_TEC.pdf          CDR   
4  apollo_17  AS17_TEC.pdf          CDR   

                                            dialogue  
0         Roger. The clock has started. We have yaw.  
1  Roger; tower. Yaw's complete. We're into roll,...  
2  Roger, Geno. Looking great. Thrust good on all...  
3               Okay, babe. It 's looking good here.  
4                 Roll is complete. We are pitching.

In [7]:
dialogue = raw_dataset["dialogue"]

In [8]:
dialogue = dialogue.str.lower()
print(dialogue.head())

0           roger. the clock has started. we have yaw.
1    roger; tower. yaw's complete. we're into roll,...
2    roger, geno. looking great. thrust good on all...
3                 okay, babe. it 's looking good here.
4                   roll is complete. we are pitching.
Name: dialogue, dtype: object


In [ ]:
# Combine all text in the DataFrame into a single string
all_text = ''.join(dialogue.astype(str).values.flatten())

# Count each character
char_counts = Counter(all_text)
print(char_counts)
print(len(char_counts))

to_remove = []

for char in char_counts:
    
    if char_counts[char] < 150:
        to_remove.append(char)


Counter({' ': 2049011, 'e': 887752, 't': 803702, 'o': 726946, 'a': 631592, 'i': 495660, 'n': 478271, 'r': 429283, 'h': 414336, 's': 407060, 'l': 321261, '.': 307345, 'u': 272136, 'd': 264813, 'y': 227812, 'g': 220862, 'c': 201774, 'w': 174847, ',': 168867, 'm': 168357, 'p': 143206, 'k': 138371, 'f': 123640, 'b': 115091, "'": 107150, '-': 89039, 'v': 80173, '0': 75514, '1': 51513, '2': 44407, '_': 37795, '5': 34234, '3': 32895, '4': 28397, '?': 27432, 'j': 24275, '6': 19258, ';': 18015, '7': 17305, '8': 14368, '9': 12991, ':': 9856, 'x': 9465, '*': 8922, 'q': 7425, 'z': 6531, '(': 6245, ')': 6140, '"': 5376, '/': 4889, '!': 2861, '°': 2129, ']': 1932, '[': 1602, '%': 891, '{': 878, '}': 600, '&': 563, '·': 512, '#': 327, '$': 284, '=': 208, '¢': 146, '£': 115, '¥': 111, '<': 106, '©': 85, '±': 82, '>': 82, '+': 63, '\\': 52, '~': 32, '×': 18, '^': 15, '@': 11, '®': 7, '½': 6, '¸': 6, '|': 6, '§': 3, '¼': 1, 'ù': 1})
82
['½', '©', '@', '±', '¥', '×', '£', '<', '¢', '\\', '^', '~', '§', '

In [20]:
clean_dialogue = []
joined = ''.join(to_remove)
joined+="{}[]°"

trans = str.maketrans('', '', joined)

for line in dialogue:
    clean_dialogue.append(line.translate(trans))

In [23]:
dialogue = pd.DataFrame(clean_dialogue)

# dialogue = dialogue.sample(frac=1).reset_index(drop=True)
dialogue.head()


0
0         roger. the clock has started. we have yaw.
1  roger; tower. yaw's complete. we're into roll,...
2  roger, geno. looking great. thrust good on all...
3               okay, babe. it 's looking good here.
4                 roll is complete. we are pitching.

In [24]:
train_end = int(0.7 * len(clean_dialogue))
dev_end = int(0.85 * len(clean_dialogue))

# Split the DataFrame
train_df = dialogue.iloc[:train_end]
dev_df   = dialogue.iloc[train_end:dev_end]
test_df  = dialogue.iloc[dev_end:]

# Optional: check sizes
print(len(train_df), len(dev_df), len(test_df))



100608 21559 21560


In [25]:
out_dir = data_dir

train_df.to_csv(f"{data_dir}/parsed_data/train_nasa.csv", index=False)
dev_df.to_csv(f"{data_dir}/parsed_data/dev_nasa.csv", index=False)
test_df.to_csv(f"{data_dir}/parsed_data/test_nasa.csv", index=False)

In [26]:
star_trek_dialogue = pd.read_json(f"{data_dir}/raw_data/star_trek_raw_lines.json")
star_trek_dialogue = star_trek_dialogue["message"].str.lower()
star_trek_dialogue.replace('', np.nan, inplace=True)
star_trek_dialogue.dropna(inplace=True) 

In [27]:
star_trek_dialogue.head()

0                                   check the circuit.
1                                  all operating, sir.
2    it can't be the screen then. definitely someth...
3                        it could be these meteorites.
4    no, it's something else. there's still somethi...
Name: message, dtype: object

In [28]:
train_end = int(0.7 * len(star_trek_dialogue))
dev_end = int(0.85 * len(star_trek_dialogue))

# Split the DataFrame
train_df = star_trek_dialogue.iloc[:train_end]
dev_df   = star_trek_dialogue.iloc[train_end:dev_end]
test_df  = star_trek_dialogue.iloc[dev_end:]

# Optional: check sizes
print(len(train_df), len(dev_df), len(test_df))

189079 40517 40517


In [29]:
train_df.to_csv(f"{data_dir}/parsed_data/train_trek.csv", index=False)
dev_df.to_csv(f"{data_dir}/parsed_data/dev_trek.csv", index=False)
test_df.to_csv(f"{data_dir}/parsed_data/test_trek.csv", index=False)